In [1]:
import pandas as pd

In [75]:
data = pd.ExcelFile('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_prices_sales_revenue.xlsx')


In [158]:
("_" not in cols[7])

True

In [ ]:
c

In [168]:
df = data.parse(sheetname=data.sheet_names[0], skiprows=0).head(10)

# make a list of the header row and strip up to the 4th letter. This is the location and year information
cols1 = list(df.columns)
cols1 = [str(x).lower() for x in cols1]

# make another list of the first row,this is the age group information
# we need to preserve this information in the column name when we reshape the data 
cols2 = list(df.iloc[0,:])
cols2 = [str(x).lower() for x in cols2]

cols3 = list(df.iloc[1,:])
cols3 = [str(x).lower().replace(" ","_") for x in cols3]

# now join the two lists to make a combined column name which preserves our location, year and age-group information
cols = [x+"_"+y for x,y in zip(cols1,cols2)]
name = ""
for i,x in enumerate(cols):
    if "unnamed" in x:
        x = x.split('_')[-1]
        cols[i] = x
    if "_" in x:
        name = x.split("_")[0]
        continue
    elif ("_" not in x):
        cols[i] = name+"_" + x
cols[0:4] = cols3[0:4]
# Assign new column names to the dataframe
df.columns = cols
df.drop([0,1], inplace=True)


In [73]:
# Import electricity price data (annual)

price_data = pd.read_excel('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_prices_sales_revenue.xlsx')
price_data.columns = price_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# Import generator data (survey of all generators in 2018)

gen_data = pd.read_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/eia_data/eia8602018/merged_new.csv')
gen_data = gen_data.drop(gen_data.columns[gen_data.columns.str.contains("Unnamed")], axis=1)

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf,,,,R'

In [72]:
pd.read_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_prices_sales_revenue.csv')

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,RESIDENTIAL,Unnamed: 5,Unnamed: 6,Unnamed: 7,COMMERCIAL,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,OTHER,Unnamed: 21,Unnamed: 22,Unnamed: 23,TOTAL,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,NaN,NaN,NaN,NaN,Revenue,Sales,Customers,Price,Revenue,Sales,...,Customers,Price,Revenue,Sales,Customers,Price,Revenue,Sales,Customers,Price
1,Year,Month,State,Data Status,Thousand Dollars,Megawatthours,Count,Cents/kWh,Thousand Dollars,Megawatthours,...,Count,Cents/kWh,Thousand Dollars,Megawatthours,Count,Cents/kWh,Thousand Dollars,Megawatthours,Count,Cents/kWh
2,1990,1,AK,Final,"17,477","181,752",.,9.62,"15,818","178,259",...,.,.,"1,918","14,816",.,12.95,"38,848","416,438",.,9.33
3,1990,1,AL,Final,"123,332","2,070,093",.,5.96,"54,495","820,415",...,.,.,"2,851","53,954",.,5.28,"268,440","5,044,008",.,5.32
4,1990,1,AR,Final,"72,506","1,026,320",.,7.06,"30,966","474,988",...,.,.,"3,117","47,600",.,6.55,"143,635","2,331,399",.,6.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18409,2020,1,WA,Preliminary,"380,876","4,037,463","3,141,648",9.43,"228,884","2,596,890",...,5,9.56,.,.,.,.,"703,273","8,611,835","3,574,790",8.17
18410,2020,1,WI,Preliminary,"303,711","2,122,930","2,766,310",14.31,"209,448","1,966,289",...,2,13.97,.,.,.,.,"653,614","5,975,758","3,134,809",10.94
18411,2020,1,WV,Preliminary,"123,713","1,141,289","859,403",10.84,"57,706","658,511",...,0,0.00,.,.,.,.,"255,001","3,045,811","1,016,452",8.37
18412,2020,1,WY,Preliminary,"32,007","305,050","278,114",10.49,"29,288","320,225",...,0,0.00,.,.,.,.,"116,883","1,480,941","347,281",7.89


In [66]:
price_data[price_data['industry_sector_category']=="Total Electric Industry"]

,year,state,industry_sector_category,residential,commercial,industrial,transportation,other,total
0,2018,AK,Total Electric Industry,21.94,18.58,17.10,0.00,NaN,19.36
1,2018,AL,Total Electric Industry,12.18,11.24,6.01,0.00,NaN,9.63
2,2018,AR,Total Electric Industry,9.81,7.75,5.64,11.35,NaN,7.78
3,2018,AZ,Total Electric Industry,12.77,10.64,6.55,10.02,NaN,10.85
4,2018,CA,Total Electric Industry,18.84,16.34,13.20,8.64,NaN,16.58
...,...,...,...,...,...,...,...,...,...
5306,1970,WA,Total Electric Industry,1.06,1.10,0.31,NaN,0.0,0.67
5307,1970,WI,Total Electric Industry,2.30,2.48,1.42,NaN,0.0,2.04
5308,1970,WV,Total Electric Industry,2.19,1.98,0.89,NaN,0.0,1.35
5309,1970,WY,Total Electric Industry,2.56,1.80,1.03,NaN,0.0,1.48
